When comparing darts arima prediction we observe the following:
* the arima model that was trained on the maximum resampling method forecasts a lower prediction than the arima model that was trained on the minimum resampling method
* the arimax model that was trained on the maximum resampling method and the median resampling method as exog. series, the predicted value reduces even more in comparison to the arima model that was trained on the maximum resampling method

In [1]:
import pickle
import time
import numpy

starttime = time.time()
print('Start reading the input file.')

# Run for the one or the other
# read dict with test chunk 200349.0_220045.0_2139-06-01 17:30:00 and steady train size 30
input_file = open('dict_of_chunk_series_with_test_and_train_test_chunk.pickle', 'rb')
dict_of_chunk_series_with_test_and_train = pickle.load(input_file)
input_file.close()

endtime = round(((time.time() - starttime) / 60), 5)
print('Reading of the input file completed after '+str(endtime)+' minutes.')

Start reading the input file.
Reading of the input file completed after 0.01923 minutes.


In [2]:
# generate example of two iterations for chunk 200349.0_220045.0_2139-06-01 17:30:00
import copy
dict_of_chunk_series_with_test_and_train_red = {}
dict_of_chunk_series_with_test_and_train_red['200349.0_220045.0_2139-06-01 17:30:00'] = {}
dict_of_chunk_series_with_test_and_train_red['200349.0_220045.0_2139-06-01 17:30:00'][29] = copy.deepcopy(dict_of_chunk_series_with_test_and_train['200349.0_220045.0_2139-06-01 17:30:00'][29])
dict_of_chunk_series_with_test_and_train_red['200349.0_220045.0_2139-06-01 17:30:00'][30] = copy.deepcopy(dict_of_chunk_series_with_test_and_train['200349.0_220045.0_2139-06-01 17:30:00'][30])

## High Alarm Forecast - Darts ARIMA


In [3]:
#perform for high alarm
import pandas as pd
from darts import TimeSeries

# generate train list for darts for one iteration
chunk = '200349.0_220045.0_2139-06-01 17:30:00'
chunk_iteration = 29
current_train_list_high = dict_of_chunk_series_with_test_and_train_red[chunk][chunk_iteration]["TRAIN_LIST_MAX"] 

#adapt for darts - convert to TimeSeries
current_train_df_high = current_train_list_high.to_frame()
current_train_df_high.reset_index(level=0,inplace=True)
start = 'Jan 1, 1970 00:00'
current_train_df_high['timestamp'] = pd.to_datetime(current_train_df_high.index, origin=start, unit='h')
current_train_list_darts_timeseries_high = TimeSeries.from_dataframe(current_train_df_high, 'timestamp', 'train_list_max')

In [ ]:
# inspect time series - high
current_train_list_darts_timeseries_high

In [7]:
TEST = 1
from darts.models import AutoARIMA
arima_high = AutoARIMA(seasonal=False, suppress_warnings=True, error_action='ignore')
arima_high.fit(current_train_list_darts_timeseries_high)
forecast_arima_high = arima_high.predict(TEST)

In [8]:
forecast_arima_high
# darts arima forecast for high: 62.9

                             0
1970-01-02 06:00:00  62.907108
Freq: H

## Low Alarm Forecast - Darts ARIMA

In [9]:
# perform for low alarm
import pandas as pd
from darts import TimeSeries

# generate train list for darts for one iteration
chunk = '200349.0_220045.0_2139-06-01 17:30:00'
chunk_iteration = 29
current_train_list_low = dict_of_chunk_series_with_test_and_train_red[chunk][chunk_iteration]["TRAIN_LIST_MIN"] 

#adapt for darts - convert to TimeSeries
current_train_df_low = current_train_list_low.to_frame()
current_train_df_low.reset_index(level=0,inplace=True)
start = 'Jan 1, 1970 00:00'
current_train_df_low['timestamp'] = pd.to_datetime(current_train_df_low.index, origin=start, unit='h')
current_train_list_darts_timeseries_low = TimeSeries.from_dataframe(current_train_df_low, 'timestamp', 'train_list_min')


In [ ]:
# inspect time series
current_train_list_darts_timeseries_low

In [11]:
TEST = 1
from darts.models import AutoARIMA
arima_low = AutoARIMA(seasonal=False, suppress_warnings=True, error_action='ignore')
arima_low.fit(current_train_list_darts_timeseries_low)
forecast_arima_low = arima_low.predict(TEST)

In [12]:
forecast_arima_low
# darts arima forecast for low: 73.0

                        0
1970-01-02 06:00:00  73.0
Freq: H

## High Alarm Forecast - Darts ARIMAX

In [13]:
# test with exog
chunk = '200349.0_220045.0_2139-06-01 17:30:00'
chunk_iteration = 29
current_train_list_high = dict_of_chunk_series_with_test_and_train_red[chunk][chunk_iteration]["TRAIN_LIST_MAX"]
current_train_list_exog_high = dict_of_chunk_series_with_test_and_train_red[chunk][chunk_iteration]["TRAIN_LIST_MEDIAN"]
current_test_list_exog_high = dict_of_chunk_series_with_test_and_train_red[chunk][chunk_iteration]["TEST_LIST_MEDIAN"]

#adapt for darts - convert to TimeSeries - Train List Exog
current_train_df_exog_high = current_train_list_exog_high.to_frame()
current_train_df_exog_high.reset_index(level=0,inplace=True)
start = 'Jan 1, 1970 00:00'
current_train_df_exog_high['timestamp'] = pd.to_datetime(current_train_df_exog_high.index, origin=start, unit='h')
current_train_list_darts_timeseries_exog_high = TimeSeries.from_dataframe(current_train_df_exog_high, 'timestamp', 'train_list_median')

#adapt for darts - convert to TimeSeries - Test List Exog
current_test_df_exog_high = current_test_list_exog_high.to_frame()
current_test_df_exog_high.reset_index(level=0,inplace=True)
start = 'Jan 1, 1970 00:00'
current_test_df_exog_high['timestamp'] = pd.to_datetime(current_test_df_exog_high.index, origin=start, unit='h')
current_test_list_darts_timeseries_exog_high = TimeSeries.from_dataframe(current_test_df_exog_high, 'timestamp', 'test_list_median', freq='H')

from darts.models import AutoARIMA
arimax_high = AutoARIMA(seasonal=False, suppress_warnings=True, error_action='ignore')
arimax_high.fit(current_train_list_darts_timeseries_high, exog=current_train_list_darts_timeseries_exog_high)
forecast_arimax_high = arimax_high.predict(TEST, exog=current_test_list_darts_timeseries_exog_high)

In [14]:
forecast_arimax_high
# darts arimax forecast for high: 60.6

                            0
1970-01-02 06:00:00  60.59797
Freq: H

## Compare to pmdarima

In [15]:

#perform for high alarm
import pandas as pd

# generate train list 
chunk = '200349.0_220045.0_2139-06-01 17:30:00'
chunk_iteration = 29
current_train_list_high = dict_of_chunk_series_with_test_and_train_red[chunk][chunk_iteration]["TRAIN_LIST_MAX"] 

In [16]:
import pmdarima as pm

TEST = 1

arima_high_pm = pm.auto_arima(current_train_list_high, seasonal=False, suppress_warnings=True, error_action='ignore')
forecast_arima_high_pm = pd.Series(arima_high_pm.predict(TEST))

In [17]:
forecast_arima_high_pm
# darts arima forecast for high: 62.9
# pmd arima forecast for high: 62.9

0    62.907108
dtype: float64

In [18]:
#perform for low alarm
import pandas as pd

# generate train list 
chunk = '200349.0_220045.0_2139-06-01 17:30:00'
chunk_iteration = 29
current_train_list_low = dict_of_chunk_series_with_test_and_train_red[chunk][chunk_iteration]["TRAIN_LIST_MIN"] 

In [19]:
import pmdarima as pm

TEST = 1

arima_low_pm = pm.auto_arima(current_train_list_low, seasonal=False, suppress_warnings=True, error_action='ignore')
forecast_arima_low_pm = pd.Series(arima_low_pm.predict(TEST))

In [20]:
forecast_arima_low_pm
# darts arima forecast for low: 73.0
# pmd arima forecast for low: 73.0

0    73.0
dtype: float64

In [23]:
# test with exog
chunk = '200349.0_220045.0_2139-06-01 17:30:00'
chunk_iteration = 29
current_train_list_high = dict_of_chunk_series_with_test_and_train_red[chunk][chunk_iteration]["TRAIN_LIST_MAX"]
current_train_list_exog_high = dict_of_chunk_series_with_test_and_train_red[chunk][chunk_iteration]["TRAIN_LIST_MEDIAN"].values.reshape(-1, 1)
current_test_list_exog_high = dict_of_chunk_series_with_test_and_train_red[chunk][chunk_iteration]["TEST_LIST_MEDIAN"].values.reshape(-1, 1)

In [24]:
import pmdarima as pm

TEST = 1
arimax_high_pm = pm.auto_arima(current_train_list_high, X=current_train_list_exog_high,seasonal=False, suppress_warnings=True, error_action='ignore')
forecast_arimax_high_pm = pd.Series(arimax_high_pm.predict(TEST, X=current_test_list_exog_high))

In [25]:
forecast_arimax_high_pm
# darts arimax forecast for high: 60.6
# pmd arimax forecast for high: 60.6

0    60.59797
dtype: float64